In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")

In [3]:
stats

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,0,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,...,0.0,0.0,Phoenix Suns,41,41,0.500,23.0,104.3,104.0,0.28
1,1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,...,0.0,0.0,Miami Heat,50,32,0.610,6.0,88.9,86.6,1.73
2,2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,...,0.0,0.0,Dallas Mavericks,24,58,0.293,40.0,90.6,97.0,-6.47
3,3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,...,0.0,0.0,Dallas Mavericks,20,62,0.244,42.0,91.4,97.5,-6.33
4,4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,...,0.0,0.0,Dallas Mavericks,19,31,0.380,18.0,91.6,94.0,-2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16046,16046,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,...,0.0,0.0,Detroit Pistons,50,32,0.610,0.0,91.4,87.7,2.97
16047,16047,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,...,0.0,0.0,Detroit Pistons,50,32,0.610,0.0,94.3,92.2,1.69
16048,16048,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,...,0.0,0.0,Los Angeles Clippers,37,45,0.451,25.0,95.7,96.5,-0.47
16049,16049,Željko Rebrača,C,33,LAC,29,2,14.2,1.8,3.3,...,0.0,0.0,Los Angeles Clippers,47,35,0.573,7.0,97.2,95.6,1.75


In [4]:
del stats["Unnamed: 0"]

In [5]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          68
3P            0
3PA           0
3P%        2146
2P            0
2PA           0
2P%         118
eFG%         68
FT            0
FTA           0
FT%         590
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W             0
L             0
W/L%          0
GB            0
PS/G          0
PA/G          0
SRS           0
dtype: int64

In [6]:
stats = stats.fillna(0)

In [7]:
predictors = [ 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 
       'W/L%', 'GB', 'PS/G','PA/G', 'SRS']

In [8]:
for col in predictors:
    # This will find any value that cannot be converted to a number
    bad_rows = stats[pd.to_numeric(stats[col], errors='coerce').isna() & stats[col].notna()]
    if not bad_rows.empty:
        print(f"Non-numeric values found in column '{col}':")
        print(bad_rows[[col, "Player", "Year"]].head())

In [9]:
train = stats[stats["Year"] < 2025]
test = stats[stats["Year"] == 2025]

In [10]:

from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [11]:
reg.fit(train[predictors], train["Share"])

,alpha,0.1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [12]:
predictions = reg.predict((test[predictors]))

In [13]:
predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)

In [14]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [15]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
13694,Shai Gilgeous-Alexander,0.913,0.173818
11667,Nikola Jokić,0.787,0.195354
5675,Giannis Antetokounmpo,0.470,0.232334
7301,Jayson Tatum,0.311,0.115932
4478,Donovan Mitchell,0.074,0.074454
9686,LeBron James,0.016,0.110306
821,Anthony Edwards,0.012,0.076318
2051,Cade Cunningham,0.012,0.088512
14052,Stephen Curry,0.002,0.070958
6831,James Harden,0.001,0.084865


In [16]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combination["Share"], combination["predictions"])

0.0027635230762485382

In [17]:
combination = combination.sort_values("Share", ascending=False)
combination["Rk"] = list(range(1, combination.shape[0] + 1))










In [18]:
combination.head(10)

,Player,Share,predictions,Rk
13694,Shai Gilgeous-Alexander,0.913,0.173818,1
11667,Nikola Jokić,0.787,0.195354,2
5675,Giannis Antetokounmpo,0.470,0.232334,3
7301,Jayson Tatum,0.311,0.115932,4
4478,Donovan Mitchell,0.074,0.074454,5
9686,LeBron James,0.016,0.110306,6
821,Anthony Edwards,0.012,0.076318,7
2051,Cade Cunningham,0.012,0.088512,8
14052,Stephen Curry,0.002,0.070958,9
6831,James Harden,0.001,0.084865,10


In [19]:
combination = combination.sort_values("predictions", ascending=False)
combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))










In [20]:
combination.head(10)

,Player,Share,predictions,Rk,Predicted_Rk
5675,Giannis Antetokounmpo,0.470,0.232334,3,1
11667,Nikola Jokić,0.787,0.195354,2,2
13694,Shai Gilgeous-Alexander,0.913,0.173818,1,3
15990,Zion Williamson,0.000,0.124608,471,4
7301,Jayson Tatum,0.311,0.115932,4,5
15459,Victor Wembanyama,0.000,0.111844,17,6
9686,LeBron James,0.016,0.110306,6,7
11936,Paolo Banchero,0.000,0.101039,380,8
8957,Kevin Durant,0.000,0.094082,287,9
8615,Karl-Anthony Towns,0.000,0.093386,272,10


In [21]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions,Rk,Predicted_Rk
13694,Shai Gilgeous-Alexander,0.913,0.173818,1,3
11667,Nikola Jokić,0.787,0.195354,2,2
5675,Giannis Antetokounmpo,0.470,0.232334,3,1
7301,Jayson Tatum,0.311,0.115932,4,5
4478,Donovan Mitchell,0.074,0.074454,5,19
9686,LeBron James,0.016,0.110306,6,7
2051,Cade Cunningham,0.012,0.088512,8,12
821,Anthony Edwards,0.012,0.076318,7,18
14052,Stephen Curry,0.002,0.070958,9,24
6637,Jalen Brunson,0.001,0.088015,12,13


In [22]:
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found +=1
            ps.append(found/seen)
        seen +=1
    return sum(ps)/len(ps)



In [23]:
find_ap(combination)

0.8126315789473685

In [24]:
years = list(range(1991, 2025))

In [25]:
aps = []
all_predictions = []
for year in years[5:]:
    train = stats[stats["Year"] < year]
    test = stats[stats["Year"] == year]
    reg.fit(train[predictors], train["Share"])
    predictions = reg.predict((test[predictors]))
    predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
    combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
    all_predictions.append(combination)
    aps.append(find_ap(combination))

In [26]:
sum(aps)/len(aps)

0.7205352809509212

In [1]:
def add_ranks(combination):
    # Add actual ranks
    combination = combination.sort_values("Share", ascending=False)
    combination["Rk"] = list(range(1, combination.shape[0] + 1))
    combination['Rk'] = combination['Rk'].apply(lambda x: x if x <= 10 else 11)
    # Add predicted ranks
    combination = combination.sort_values("predictions", ascending=False)
    combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))
    combination['Predicted_Rk'] = combination['Predicted_Rk'].apply(lambda x: x if x <= 10 else 11)
    combination["Diff"] = combination["Rk"] - combination["Predicted_Rk"]
    return combination


In [28]:
ranking =  add_ranks(all_predictions[1])
ranking[ranking["Rk"] < 6].sort_values("Diff", ascending=False)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
8603,Karl Malone,0.857,0.192318,1,2,-1
10956,Michael Jordan,0.832,0.167629,2,3,-1
5781,Grant Hill,0.327,0.128646,3,6,-3
14628,Tim Hardaway,0.207,0.059984,4,20,-16
5700,Glen Rice,0.117,0.033110,5,53,-48


In [57]:
def backtest(stats, model, year, predictors):
    aps = []
    all_predictions = []
    for year in years[5:]:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]
        model.fit(train[predictors], train["Share"])
        predictions = model.predict((test[predictors]))
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps)/len(aps), aps, pd.concat(all_predictions)
    

In [30]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [31]:
mean_ap

0.7205352809509212

In [32]:
all_predictions[all_predictions["Rk"] <= 5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
7138,Jason Kidd,0.712,0.028209,2,52,-50
5700,Glen Rice,0.117,0.033110,5,53,-48
14174,Steve Nash,0.839,0.034100,1,45,-44
12150,Peja Stojaković,0.228,0.036269,4,38,-34
14169,Steve Nash,0.739,0.054128,1,34,-33
7704,Joakim Noah,0.258,0.046968,4,37,-33
2420,Chauncey Billups,0.344,0.052698,5,35,-30
2658,Chris Paul,0.138,0.072294,5,33,-28
14170,Steve Nash,0.785,0.074421,2,21,-19
14628,Tim Hardaway,0.207,0.059984,4,20,-16


In [33]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)

,0,1
13,0.119161,eFG%
29,0.031201,W/L%
18,0.030212,DRB
17,0.016535,ORB
10,0.015529,2P
21,0.012219,STL
22,0.010264,BLK
15,0.010186,FTA
12,0.010172,2P%
25,0.007808,PTS


In [34]:
stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())

C:\Users\Daniel\AppData\Local\Temp\ipykernel_31048\2022889540.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())


In [39]:
stat_ratios = stat_ratios.reset_index(level=0, drop=True)

In [40]:
stat_ratios

,PTS,AST,STL,BLK,3P,Year
10,1.013334,0.420714,0.961127,0.673469,0.508587,1.0
13,0.979927,1.168650,0.549215,0.448980,0.000000,1.0
29,0.222711,0.607698,0.274608,0.000000,0.000000,1.0
168,0.200440,0.093492,0.686519,0.448980,0.000000,1.0
171,0.634726,0.420714,0.686519,0.000000,0.508587,1.0
...,...,...,...,...,...,...
15923,1.011217,0.477308,0.740067,3.159363,0.277799,1.0
15971,0.351728,0.190923,0.592053,1.701195,0.277799,1.0
15989,1.099149,0.620501,1.480133,0.972112,1.481594,1.0
15990,2.703906,2.529734,1.776160,2.187251,0.092600,1.0


In [41]:
stats[["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

In [44]:
predictors += ["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [45]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [46]:
mean_ap

0.7286748534369373

In [47]:
stats["NPos"] = stats["Pos"].astype("category").cat.codes

In [49]:
stats["NTm"] = stats["Tm"].astype("category").cat.codes

In [51]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators= 50, random_state=1, min_samples_split=5)



In [58]:
mean_ap, aps, all_predictions = backtest(stats, rf, years[28:], predictors)

In [59]:
mean_ap

0.742378285273556

In [60]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[28:], predictors)
mean_ap

0.7286748534369373